# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr

In [2]:
# Environment setup and API configuration

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

# Create clients for different models
openai = OpenAI()

# Ollama client (uses OpenAI-compatible endpoint)
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

OpenAI API Key exists and begins sk-proj-


In [3]:
# System prompt for technical expertise

system_prompt = """
You are a helpful technical assistant with expertise in:
- Software Engineering and System Design
- Software Architecture Patterns
- Programming Languages and Frameworks
- Distributed Systems and Scalability
- Database Design and Optimization
- DevOps and Cloud Computing

Structure your explanations to include:
1. **What**: Clear explanation of the concept or code
2. **Why**: Reasoning behind the approach or design
3. **Use Cases**: Common scenarios where this is applied
4. **Considerations**: Important trade-offs, alternatives, or best practices

Provide clear, detailed, and accurate technical explanations.
"""

In [4]:
# Available models configuration

MODELS = {
    "GPT-4o-mini": {"client": openai, "model": "gpt-4o-mini"},
    "GPT-4.1-mini": {"client": openai, "model": "gpt-4.1-mini"},
    "Llama 3.2": {"client": ollama, "model": "llama3.2"},
    "DeepSeek R1 1.5B": {"client": ollama, "model": "deepseek-r1:1.5b"},
}

In [5]:
# Tool: Search for code examples or documentation references (simulated)

def search_code_examples(topic):
    """
    Simulates searching for code examples related to a topic.
    In a real application, this could query a code repository, Stack Overflow, or documentation.
    """
    print(f"TOOL CALLED: Searching code examples for '{topic}'")
    
    # Simulated code examples database
    examples = {
        "microservices": "Example: Use API Gateway pattern, implement service discovery with Consul/Eureka, use message queues for async communication.",
        "caching": "Example: Redis for in-memory caching, Memcached for distributed caching, implement cache-aside pattern.",
        "authentication": "Example: JWT tokens for stateless auth, OAuth2 for third-party integration, implement refresh token rotation.",
        "database": "Example: Use indexing for query optimization, implement connection pooling, consider read replicas for scaling.",
        "api": "Example: RESTful design with proper HTTP verbs, versioning in URLs, implement rate limiting and pagination.",
    }
    
    # Simple keyword matching
    for key, example in examples.items():
        if key in topic.lower():
            return example
    
    return "No specific code examples found for this topic, but I can still help explain it!"

In [6]:
# Define tool for OpenAI function calling

search_function = {
    "name": "search_code_examples",
    "description": "Search for code examples and best practices related to a technical topic or concept.",
    "parameters": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The technical topic or concept to search examples for (e.g., 'microservices', 'caching', 'authentication')",
            },
        },
        "required": ["topic"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": search_function}]

In [7]:
# Handle tool calls from the model

def handle_tool_calls(message):
    """
    Process tool calls requested by the model and return responses.
    """
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "search_code_examples":
            arguments = json.loads(tool_call.function.arguments)
            topic = arguments.get('topic')
            example_details = search_code_examples(topic)
            responses.append({
                "role": "tool",
                "content": example_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [8]:
# Main chat function with streaming support

def chat(message, history, model_name):
    """
    Main chat function that:
    1. Takes user message and conversation history
    2. Calls selected model with tools
    3. Handles tool calls if needed
    4. Streams the response back
    """
    # Convert Gradio history format to API messages format
    history_messages = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build complete messages list
    messages = [
        {"role": "system", "content": system_prompt}
    ] + history_messages + [
        {"role": "user", "content": message}
    ]
    
    # Get the selected model client and model name
    model_config = MODELS.get(model_name, MODELS["GPT-4o-mini"])
    client = model_config["client"]
    model = model_config["model"]
    
    # Make API call with tools (only for OpenAI/GPT models)
    use_tools = client == openai
    
    if use_tools:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            stream=True
        )
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
    
    # Stream the response
    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            partial_message += chunk.choices[0].delta.content
            yield partial_message
        
        # Handle tool calls (non-streaming for simplicity)
        if hasattr(chunk.choices[0].delta, 'tool_calls') and chunk.choices[0].delta.tool_calls:
            # If tool call is detected, we need to handle it
            # For simplicity in streaming, we'll just notify
            partial_message += "\n\n*[Searching code examples...]*\n\n"
            yield partial_message

In [9]:
# Alternative chat function with full tool support (non-streaming)

def chat_with_tools(message, history, model_name):
    """
    Chat function with complete tool support.
    Handles multiple tool call rounds if needed.
    """
    # Convert history
    history_messages = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build messages
    messages = [
        {"role": "system", "content": system_prompt}
    ] + history_messages + [
        {"role": "user", "content": message}
    ]
    
    # Get model config
    model_config = MODELS.get(model_name, MODELS["GPT-4o-mini"])
    client = model_config["client"]
    model = model_config["model"]
    
    # Only use tools with OpenAI models
    if client == openai:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        
        # Handle tool calls if present
        while response.choices[0].finish_reason == "tool_calls":
            assistant_message = response.choices[0].message
            tool_responses = handle_tool_calls(assistant_message)
            messages.append(assistant_message)
            messages.extend(tool_responses)
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            )
    else:
        # For Ollama models, no tool support
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
    
    return response.choices[0].message.content

In [ ]:
# Test the basic functionality before creating UI

test_question = "What is microservices architecture and when should I use it?"
test_history = []

response = chat_with_tools(test_question, test_history, "GPT-4o-mini")
display(Markdown(response))

## Gradio UI with Streaming and Model Selection

Now we'll create a complete Gradio interface that includes:
- Chat interface with history
- Model selection dropdown
- Streaming responses for better UX
- Tool integration (code example search)

In [ ]:
# Create the Gradio UI with custom blocks for model selection

with gr.Blocks(title="Technical Q&A Assistant") as demo:
    gr.Markdown(
        """
        # Technical Q&A Assistant
        
        Ask me anything about software engineering, system design, and architecture!

        """
    )
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=500,
                type="messages",
                label="Conversation"
            )
        
        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=list(MODELS.keys()),
                value="GPT-4o-mini",
                label="Select Model",
                info="Choose which AI model to use"
            )
            
            gr.Markdown(
                """
                ### Sample Questions:
                
                - "Explain microservices architecture"
                - "What is the difference between SQL and NoSQL?"
                - "How does JWT authentication work?"
                - "Explain the MVC pattern"
                - "What are design patterns in software engineering?"
                """
            )
    
    with gr.Row():
        message = gr.Textbox(
            label="Ask a technical question:",
            placeholder="Type your question here...",
            lines=2
        )
    
    with gr.Row():
        submit_btn = gr.Button("Submit", variant="primary")
        clear_btn = gr.ClearButton([message, chatbot], value="Clear Chat")
    
    # Event handlers
    def add_message(user_message, history):
        """Add user message to chat history"""
        return "", history + [{"role": "user", "content": user_message}]
    
    def bot_response(history, model_name):
        """Generate and stream bot response"""
        user_message = history[-1]["content"]
        bot_history = history[:-1]  # Exclude the last user message
        
        # Use streaming chat function
        partial_response = ""
        for partial in chat(user_message, bot_history, model_name):
            partial_response = partial
            yield history + [{"role": "assistant", "content": partial_response}]
    
    # Connect events
    message.submit(
        add_message,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        bot_response,
        inputs=[chatbot, model_dropdown],
        outputs=chatbot
    )
    
    submit_btn.click(
        add_message,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        bot_response,
        inputs=[chatbot, model_dropdown],
        outputs=chatbot
    )

# Launch the interface
demo.launch()

## BONUS: Enhanced version with Audio Support

This advanced version includes:
- Audio input (speech-to-text)
- Audio output (text-to-speech)
- All the features from above

Note: This uses OpenAI's Whisper for transcription and TTS for audio generation.

In [14]:
# Audio transcription function

def transcribe_audio(audio_path):
    """
    Transcribe audio file to text using Whisper.
    """
    if audio_path is None:
        return None
    
    with open(audio_path, "rb") as audio_file:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    return transcript.text

In [15]:
# Text-to-speech function

def text_to_speech(text):
    """
    Convert text response to audio.
    """
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",  # Options: alloy, echo, fable, onyx, nova, shimmer
        input=text
    )
    return response.content

In [ ]:
# Enhanced UI with Audio Support

with gr.Blocks(title="Technical Q&A Assistant - Enhanced") as demo_audio:
    gr.Markdown(
        """
        # Technical Q&A Assistant with Audio
        
        Ask questions via text or voice, and get audio responses!
        """
    )
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                height=400,
                type="messages",
                label="Conversation"
            )
            audio_output = gr.Audio(
                label="Audio Response",
                autoplay=True
            )
        
        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=list(MODELS.keys()),
                value="GPT-4o-mini",
                label="Select Model"
            )
            
            enable_audio = gr.Checkbox(
                label="Enable Audio Output",
                value=True,
                info="Generate speech for responses"
            )
    
    with gr.Row():
        with gr.Column(scale=3):
            message = gr.Textbox(
                label="Type your question:",
                placeholder="Or use audio input below...",
                lines=2
            )
        with gr.Column(scale=1):
            audio_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="Or speak your question:"
            )
    
    with gr.Row():
        submit_btn = gr.Button("Submit", variant="primary")
        clear_btn = gr.ClearButton([message, chatbot, audio_output], value="Clear All")
    
    # Event handlers
    def process_audio(audio_path, current_message):
        """Transcribe audio and add to text input"""
        if audio_path:
            transcribed = transcribe_audio(audio_path)
            return transcribed if transcribed else current_message
        return current_message
    
    def add_message_enhanced(user_message, history):
        """Add user message to chat"""
        if not user_message:
            return "", history, None
        return "", history + [{"role": "user", "content": user_message}], None
    
    def bot_response_enhanced(history, model_name, audio_enabled):
        """Generate response with optional audio"""
        user_message = history[-1]["content"]
        bot_history = history[:-1]
        
        # Get text response (with tools if GPT)
        response_text = chat_with_tools(user_message, bot_history, model_name)
        
        # Update chat history
        updated_history = history + [{"role": "assistant", "content": response_text}]
        
        # Generate audio if enabled and using OpenAI model
        audio = None
        if audio_enabled and model_name.startswith("GPT"):
            audio = text_to_speech(response_text)
        
        return updated_history, audio
    
    # Audio input triggers transcription
    audio_input.change(
        process_audio,
        inputs=[audio_input, message],
        outputs=message
    )
    
    # Submit handlers
    message.submit(
        add_message_enhanced,
        inputs=[message, chatbot],
        outputs=[message, chatbot, audio_input]
    ).then(
        bot_response_enhanced,
        inputs=[chatbot, model_dropdown, enable_audio],
        outputs=[chatbot, audio_output]
    )
    
    submit_btn.click(
        add_message_enhanced,
        inputs=[message, chatbot],
        outputs=[message, chatbot, audio_input]
    ).then(
        bot_response_enhanced,
        inputs=[chatbot, model_dropdown, enable_audio],
        outputs=[chatbot, audio_output]
    )

# Launch with authentication (optional)
demo_audio.launch()